<img src="img/python-logo-notext.svg"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;"><b>Concurrency</b></div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>



# Concurrency

Definition by Leslie Lamport (in *Time, Clocks, and the Ordering of Events*, 1976):

<blockquote>
Two events are concurrent if neither can causally affect the other.
<blockquote>

I.e., concurrent events can be executed in any order.


When is concurrency useful?

- To reduce latency and increase throughput
- To take advantage of multiple processor cores
- To perform background activities


How can concurrency be realized?

- Interleaving (time slices)
- Asynchronous processing (special case of interleaving)
- Parallel processing


How can concurrency be realized?

- Interleaving (time slices): Coroutines, ...
- Asynchronous processing (special case of interleaving): event loops, async, ...
- Parallel processing: threads, processes, futures, ...

But: In Python, threads usually cause interleaving rather than real parallel
processing!


## Threads

Threads are encapsulated by the `threading.Thread` class:



### Background Processing

In [ ]:
def wait_and_print():
    from time import sleep
    print("Starting...")
    sleep(10)
    print("Stopping...")

In [ ]:
from threading import Thread

my_thread = Thread(target=wait_and_print)

In [ ]:
my_thread.start()

In [ ]:
print("Hello, from main Thread!")
print("My thread is alive:", my_thread.is_alive())

In [ ]:
my_thread.join()
print("This should run only after my_thread is done.")
print("My thread is alive:", my_thread.is_alive())


### Reducing latency and increasing throughput

In [ ]:
from time import sleep
from random import random
import timeit


def simulate_processing_time(delta_time=0.1):
    sleep(random() * delta_time + delta_time)

In [ ]:
def process_request(data, results, delta_time=0.1):
    simulate_processing_time(delta_time)
    # Is this correct?
    results.append(f"->{data}")

In [ ]:
def process_requests_sequentially(num_requests):
    results = []
    for i in range(num_requests):
        process_request(i, results)
    return results

In [ ]:
process_requests_sequentially(5)

In [ ]:
timeit.timeit(lambda: process_requests_sequentially(5), globals=globals(), number=10)

In [ ]:
timeit.timeit(lambda: process_requests_sequentially(10), globals=globals(), number=10)

In [ ]:
from threading import Thread


def process_requests_concurrently(num_requests):
    results = []
    threads = []
    for i in range(num_requests):
        thread = Thread(target=lambda: process_request(i, results))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    return results

In [ ]:
process_requests_concurrently(5)

In [ ]:
timeit.timeit(lambda: process_requests_concurrently(5), globals=globals(), number=10)

In [ ]:
timeit.timeit(lambda: process_requests_concurrently(10), globals=globals(), number=10)

In [ ]:
timeit.timeit(lambda: process_requests_concurrently(100), globals=globals(), number=10)

In [ ]:
class MyThread(Thread):
    # Note `run()`is overridden, not `start()`!
    def run(self) -> None:
        # noinspection PyUnresolvedReferences
        process_request(*self._args, **self._kwargs)

In [ ]:
def process_requests_concurrently_2(num_requests):
    results = []
    threads = [MyThread(args=(i, results)) for i in range(num_requests)]
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()
    return results

In [ ]:
process_requests_concurrently_2(5)

In [ ]:
timeit.timeit(lambda: process_requests_concurrently_2(5), globals=globals(), number=10)

In [ ]:
timeit.timeit(lambda: process_requests_concurrently_2(10), globals=globals(), number=10)

In [ ]:
timeit.timeit(lambda: process_requests_concurrently_2(100), globals=globals(),
              number=10)


### Multiple Threads and the GIL

In [ ]:
def perform_computation(data, results, num_iterations=1_000_000):
    result = 0
    for i in range(num_iterations):
        result += 1
    results.append(f"->{data}: {result}")

In [ ]:
def perform_computations_sequentially(num_requests):
    results = []
    for i in range(num_requests):
        perform_computation(i, results)
    return results

In [ ]:
perform_computations_sequentially(5)

In [ ]:
timeit.timeit(lambda: perform_computations_sequentially(5), globals=globals(),
              number=10)

In [ ]:
timeit.timeit(lambda: perform_computations_sequentially(10), globals=globals(),
              number=10)

In [ ]:
def perform_computations_concurrently(num_requests):
    results = []
    threads = []
    for i in range(num_requests):
        thread = Thread(target=lambda: perform_computation(i, results))
        threads.append(thread)
        thread.start()
    for thread in threads:
        thread.join()
    return results

In [ ]:
perform_computations_concurrently(5)

In [ ]:
timeit.timeit(lambda: perform_computations_concurrently(5), globals=globals(),
              number=10)

In [ ]:
timeit.timeit(lambda: perform_computations_concurrently(10), globals=globals(),
              number=10)


In Python, only *one* Python thread is running at a time, all other threads exist but
"wait their turn". Therefore, multithreading only brings advantages when, for example,
you are waiting for input/output operations, not when multiple calculations are to be
accelerated!

## Workshop

- Notebook `workshop_410_concurrency`
- Section "Parallel Requests"


### Synchronizing Threads

Concurrent programming leads to problems that do not exist in sequential programs:

In [ ]:
def add_ones():
    global _result
    for i in range(10_000):
        tmp = _result + 1
        # if random() > 0.99:
        #     simulate_processing_time(0)
        _result = tmp

In [ ]:
from threading import Thread

_result = 0
_threads = [Thread(target=add_ones) for _ in range(100)]
for _thread in _threads:
    _thread.start()
for _thread in _threads:
    _thread.join()
print(f"\n_result = {_result}")

In [ ]:
def append_one():
    global _result_list
    for i in range(100_000):
        _result_list.append(1)

In [ ]:
from threading import Thread

_result_list = []
_threads = [Thread(target=append_one) for _ in range(100)]
for _thread in _threads:
    _thread.start()
for _thread in _threads:
    _thread.join()
print(f"\nLength of _result_list: {len(_result_list)}")


#### Barriers

A Barrier can be used to synchronize a fixed number of threads:

In [ ]:
from threading import Barrier, Thread

_barrier = Barrier(2, timeout=5)

In [ ]:
def server1():
    print("Server is starting!")
    simulate_processing_time(1.0)
    print("Server started up!")
    _barrier.wait()
    print("Server is serving!")

In [ ]:
def client1():
    print("Client is starting!")
    _barrier.wait()
    print("Client is accessing server!")

In [ ]:
_c = Thread(target=client1)
_c.start()

In [ ]:
_s = Thread(target=server1)
_s.start()

In [ ]:
_c.join()
_s.join()

In [ ]:
_s = Thread(target=server1)
_s.start()

In [ ]:
_c = Thread(target=client1)
_c.start()

In [ ]:
_s.join()
_c.join()


#### Locks

Locks are a low-level synchronization mechanism that can be used to enforce that only
a single thread can use a resource:

In [ ]:
from threading import Lock, Thread

_result_lock = Lock()

In [ ]:
def add_ones_locked():
    global _result
    for i in range(10_000):
        with _result_lock:
            tmp = _result + 1
            if random() > 0.99:
                simulate_processing_time(0)
            _result = tmp

In [ ]:
_result = 0
_threads = [Thread(target=add_ones_locked) for _ in range(100)]
for _thread in _threads:
    _thread.start()
for _thread in _threads:
    _thread.join()
print(f"\n_result = {_result}")

In [ ]:
def server2():
    _barrier.wait()
    print("Server is serving")
    print("Server is still serving")
    print("Server is serving even more data")

In [ ]:
def client2():
    _barrier.wait()
    print("Client is accessing server")
    print("Client is still accessing server")
    print("Client is taking really long to access the server")

In [ ]:
def run_tasks(task1, task2):
    thread1 = Thread(target=task2)
    thread1.start()

    thread2 = Thread(target=task1)
    thread2.start()

    thread1.join()
    thread2.join()

In [ ]:
run_tasks(server2, client2)

In [ ]:
from threading import Lock

_print_lock = Lock()

In [ ]:
def server3():
    _barrier.wait()
    try:
        _print_lock.acquire()
        simulate_processing_time()
        print("Server is serving")
        print("Server is still serving")
        print("Server is serving even more data")
    finally:
        _print_lock.release()

In [ ]:
def client3():
    _barrier.wait()
    if _print_lock.acquire(blocking=False):
        print("Client is accessing server")
        print("Client is still accessing server")
        print("Client is taking really long to access the server")
        _print_lock.release()
    else:
        print("WARNING: Could not acquire lock!!!")

In [ ]:
run_tasks(server3, client3)

In [ ]:
run_tasks(client3, server3)

In [ ]:
def server4():
    _barrier.wait()
    with _print_lock:
        print("Server is serving")
        print("Server is still serving")
        print("Server is serving even more data")

In [ ]:
def client4():
    _barrier.wait()
    with _print_lock:
        print("Client is accessing server")
        print("Client is still accessing server")
        print("Client is taking really long to access the server")

In [ ]:
run_tasks(server4, client4)

In [ ]:
run_tasks(client4, server4)


#### Condition Variables

Condition variables are a synchronization mechanism that is based on locks but
provides an additional way to coordinate threads: `wait()` and  `notify()` (or
`notify_all()`):

Typically, condition variables are used when multiple threads share a common state
and need to synchronize reading and writing the state:

- Threads that want to read the state use `wait()` or `wait_for()` to wait until
  the desired state is reached
- Threads that write the state use `notify()` or `notify_all()` to notify any
  waiting threads about the change

In [ ]:
from threading import Condition, Thread

In [ ]:
def consumer(consumer_id, cv, items):
    print(f"Consumer {consumer_id} started...", flush=True)
    with cv:
        print(f"Consumer {consumer_id} waiting...", flush=True)
        wait_succeeded = True
        while True:
            while not items and wait_succeeded:
                wait_succeeded = cv.wait(timeout=1.0)
            if not wait_succeeded:
                print(f"Consumer {consumer_id} timed out...", flush=True)
                break
            print(f"Consumer {consumer_id} starts consuming...", flush=True)
            item = items.pop()
            simulate_processing_time(0.1)
            print(f"Consumer {consumer_id} ends consuming item {item}...", flush=True)

In [ ]:
def producer(producer_id, cv, num_items, items):
    from random import randint
    print(f"Producer {producer_id} started...", flush=True)
    for _ in range(num_items):
        with cv:
            item = randint(100, 999)
            print(f"Producer {producer_id} is producing item {item}", flush=True)
            items.append(item)
            cv.notify()
            simulate_processing_time(0.05)

In [ ]:
def run_producer_consumer(num_items, num_producers=1, num_consumers=1):
    threads = []
    items = []
    cv = Condition()
    for i in range(num_consumers):
        threads.append(Thread(target=consumer, args=(i + 1, cv, items)))
    for i in range(num_producers):
        threads.append(Thread(target=producer, args=(i + 1, cv, num_items, items)))
    for thread in threads:
        thread.start()
    for thread in threads:
        thread.join()

In [ ]:
run_producer_consumer(2)

In [ ]:
run_producer_consumer(6, num_producers=1, num_consumers=3)

In [ ]:
run_producer_consumer(4, num_producers=3, num_consumers=4)

In [ ]:
from queue import Queue, Empty

In [ ]:
def producer(producer_id, q, num_items):
    print(f"Producer {producer_id} started...")
    for i in range(num_items):
        print(f"Producer {producer_id} produced item {producer_id}/{i}...")
        q.put(f"Item {producer_id}/{i}")
        simulate_processing_time(0.1)

In [ ]:
def consumer(consumer_id, q, timeout=1.0):
    print(f"Consumer {consumer_id} started...")
    try:
        while True:
            item = q.get(block=True, timeout=timeout)
            print(f"Consumer {consumer_id} starting processing of item {item}...")
            simulate_processing_time(0.2)
            print(f"Consumer {consumer_id} done processing item {item}...")
    except Empty:
        print(f"Consumer {consumer_id} timed out...")

In [ ]:
from threading import Thread
def run_producer_consumer_queue(num_items, num_producers=1, num_consumers=1):
    processes = []
    q = Queue()
    for i in range(num_consumers):
        processes.append(Thread(target=consumer, args=(i + 1, q)))
    for i in range(num_producers):
        processes.append(Thread(target=producer, args=(i + 1, q, num_items)))
    for process in processes:
        process.start()
    for process in processes:
        process.join()

In [ ]:
run_producer_consumer_queue(4)

In [ ]:
run_producer_consumer_queue(6, num_producers=1, num_consumers=3)

In [ ]:
run_producer_consumer_queue(2, num_producers=4, num_consumers=3)